## This is the updated system
1. Fetch data from NSE
2. Automatically generate prices and stop loss

## Instructions
1. See instructions for each cell
2. You need to change only the PARAMETERS, nothing else
3. Store all your data in data folder
4. All the orders you generate go into the orders folder
5. Orders for the present day are created with the filename **orders_to_place.csv** and stored in the current directory

In [1]:
# Import libraries
import pandas as pd
import os

## Parameters that could be changed

In [2]:
UNIVERSE = 'NIFTY50' # Universe to be searched
ORDER_FILENAME = 'data/cm11SEP2018bhav.csv.zip'
STOP_LOSS = 3 # Stop loss for the order
NUM_STOCKS = 5 # Number of stocks to sell
CAPITAL = 20000
LEVERAGE = 1
ORDERFILE_PREFIX = 'orders_' # Prefix file name to store

In [3]:
from utils import *
preopen = fetch_preopen_data()

In [4]:
### Don't change the below cell

In [5]:
eod = pd.read_csv(ORDER_FILENAME, parse_dates=['TIMESTAMP'],
                    usecols=range(13))
symbols = pd.read_excel('universe.xlsx', sheet_name=UNIVERSE, header=None).values.ravel()
df = eod[eod['SYMBOL'].isin(symbols)]
df = df[df['SERIES'] == "EQ"].reset_index(drop=True)

## Trading logic

There is where all your logic goes.
At present, we short the top n stocks the previous day

In [6]:
df['RET'] = (df['CLOSE']/df['PREVCLOSE']) - 1
result = df.sort_values(by='RET', ascending = False).iloc[:NUM_STOCKS]

In [7]:
trading_capital = CAPITAL * LEVERAGE
orders = result.merge(preopen, on='SYMBOL')
num_stock = len(orders)

## Pricing logic goes here

In [8]:
# I prefer a constant percentage change instead of value
# If you prefer it, make this raw and use the below cell

orders['trigger_price'] = orders['OpenPrice'] - 0.05
orders['price'] = orders['trigger_price'] - 0.05

In [9]:
# Stop loss and quantity automatically generated

orders['stop_loss'] = (orders['price'] * (1 + STOP_LOSS * 0.01)).round(2).apply(tick)
orders['qty'] = (trading_capital/num_stock/orders['price']).round()
import datetime
filename = ORDERFILE_PREFIX + datetime.datetime.today().strftime('%Y-%m-%d') + '.csv'
orders.to_csv('orders/' + filename,  index=False)

### Use the below cells to generate orders for your broker
#### I have included code for **NEST**

Most brokers and software provide an option to place basket orders. So we are going to create a basket order from our orders.  We would be using the ``create_order`` function from ``utils.py`` file.

To do this for your specific broker, do the following steps

1. Know the format of your broker; you can do this by placing a basket order and exporting it
2. The format would usually have a list of columns to be filled up. We need to fill all the columns to import our order.
3. We separate the columns into columns that are already in our dataframe and columns to be included
4. We prepare a list of matching columns and rename them
5. For new columns, we create a python dictionary with keys as column names and values as the value for the column (we assume that these columns have a single value)

All this has been done for NEST.

*Thanks @vjay for providing the necessary support*


## Parameters for order generation



In [10]:
ACCOUNTID = 'XXXXXX' # Your accountId with broker

In [11]:
# List of columns to be included in the output

cols = [
    'Segment', 'InstrumentName', 'Symbol', 'Option Type', 'Strike Price',
    'ExpiryDate', 'Price', 'Qty', 'LTP', 'Buy/Sell', 'Order Type',
    'TriggerPrice', 'Pro/Cli', 'P Type', 'Validity', 'AccountId',
    'Validity Date', 'Remarks', 'Participant code', 'Validity Time',
    'Market Proc'    
]

# These columns are common for all orders - columns with a single name
columns = {
    'Segment': 'NSE',
    'InstrumentName': 'EQ',
    'Option Type': 'NA',
    'Strike Price': 'NA',
    'ExpiryDate': 'NA',
    'LTP': 0,
    'Disclosed Qty': 0,
    'AccountId': ACCOUNTID,
    'Pro/Cli': 'CLI',
    'Validity': 'DAY',
    'P Type': 'MIS',
    'Remarks': '',
    'Validity Date': 'NA',
    'Participant code': '',
    'Validity Time': 'NA',
    'Market Proc': 'NA',
    'Order Type': 'SL'
}

# These are columns to be renamed
rename = {
    'order': 'Buy/Sell',
    'price': 'Price',
    'qty': 'Qty',
    'trigger_price': 'TriggerPrice',
    'price': 'Price' ,
    'SYMBOL': 'Symbol'
}

In [12]:
# Generating orders in the required format
entry_orders = orders.copy()
entry = create_orders(entry_orders, rename=rename, **columns)

# Exit orders order type and price to be changed
exit_orders = orders.copy()
exit_orders['order'] = 'BUY'
exit_orders['price'] = 0
exit_orders['trigger_price'] = list(stop_loss(orders['price'], 3, order='S').round(2))
columns.update({'Order Type': 'SL-M'})
exit = create_orders(exit_orders, rename=rename, **columns)

# File generation
entry.append(exit, sort=False)[cols].to_csv('orders_to_place.csv', 
                   index=False, header=False)

print('File generated')

# If you want header change header to True
# If you want to export as excel use the to_excel function

File generated
